In [91]:
import pandas as pd
import streamlit as st
st.set_page_config(layout="wide")

2025-01-08 17:10:18.866 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [30]:
import paramiko
from scp import SCPClient
import pandas as pd

# Configurações do servidor remoto
host = "145.223.30.123"
port = 22  # Porta SSH padrão
username = "root"
password = "Marcio@aynrand7"
remote_path = "/root/play/stravaapi/atividades_unicas.csv"
local_path = "atividades_unicas.csv"

def download_csv_via_scp():
    # Cria o cliente SSH
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    try:
        # Conecta ao servidor
        ssh.connect(host, port=port, username=username, password=password)

        # Cria o cliente SCP
        with SCPClient(ssh.get_transport()) as scp:
            print("Baixando arquivo...")
            scp.get(remote_path, local_path)  # Baixa o arquivo
            print(f"Arquivo baixado para {local_path}")
    except Exception as e:
        print(f"Erro ao transferir arquivo: {e}")
    finally:
        ssh.close()

def process_csv():
    try:
        # Lê o arquivo CSV local
        df = pd.read_csv(local_path)
        # Exibe as primeiras linhas do DataFrame
    except Exception as e:
        print(f"Erro ao processar o arquivo CSV: {e}")

# Executa as funções
download_csv_via_scp()
process_csv()

Baixando arquivo...
Arquivo baixado para atividades_unicas.csv
Conteúdo do CSV:
      firstname lastname                        name  distance  moving_time  \
0       Vanilda       C.                 Morning Run    4559.7         1800   
1          Neto       F.       008/365 Play DISTANCE    5000.0         1813   
2        Débora       T.                 Morning Run    4820.0         1800   
3    Mara Lúcia       C.                 Morning Run    4672.5         1813   
4  RUY  BARBOSA       R.  Parque Águas Claras  - Zx3    7941.5         2507   

   elapsed_time  total_elevation_gain type sport_type  data_atual  
0          1824                  58.0  Run        Run  2025-01-08  
1          1813                   0.0  Run        Run  2025-01-08  
2          1909                   0.0  Run        Run  2025-01-08  
3          1924                   0.0  Run        Run  2025-01-08  
4          2507                 102.7  Run        Run  2025-01-08  


In [35]:
df_strava = pd.read_csv('unique_activities_old.csv')
# df_strava = pd.read_csv('atividades_unicas.csv')
df_strava['athlete.fullname'] = df_strava['firstname'] + ' ' + df_strava['lastname']
df_strava['moving_time_minutes'] = round(df_strava['moving_time']/60, 2)

In [97]:
from datetime import datetime, timedelta


cols = ['athlete.fullname','name', 'type', 'distance',   
         'data_atual', 'distance_km', 'pace_real','moving_time_minutes'
       ]
# df_strava['moving_time_minutes'] = round(df_strava['moving_time']/60, 2)

df_strava['distance_km'] = round(df_strava['distance'] / 1000, 2)

df_strava['pace'] = df_strava['moving_time_minutes'] / df_strava['distance_km']

df_strava['avg_speed_kmh'] = round(60/df_strava['pace'], 2)


def kmh_to_min_km(speed_kmh):
    if speed_kmh > 0:  # Evitar divisão por zero
        pace = 60 / speed_kmh
        minutes = int(pace)
        seconds = int((pace - minutes) * 60)
        return f"{minutes}:{seconds:02d} min/km"
    else:
        return None  # Retorna None para velocidades inválidas

df_strava['avg_speed_kmh'] = pd.to_numeric(df_strava['avg_speed_kmh'], errors='coerce')
df_strava['pace_real'] = df_strava['avg_speed_kmh'].apply(kmh_to_min_km)

corridas = df_strava[cols]
runs = corridas.loc[corridas['type'] == 'Run' ]

# Agrupar por atleta e agregar as datas em uma lista
grouped = runs.groupby("athlete.fullname").agg(
    total_quilometros=("distance", lambda x: round(x.sum() / 1000, 2)),
    
    tempo_total=("moving_time_minutes", "sum"),
    datas=("data_atual", lambda x: list(x))
).reset_index()

# Definir o intervalo de datas
start_date = datetime.strptime('2025-01-08', '%Y-%m-%d')
end_date = datetime.strptime('2025-01-18', '%Y-%m-%d')

# Gerar uma lista de datas no intervalo
date_range = [(start_date + timedelta(days=i)).strftime('%d-%m-%y') for i in range((end_date - start_date).days + 1)]

# Verificar se cada data no intervalo está presente e criar colunas
for date in date_range:
    grouped[date] = grouped['datas'].apply(
        lambda x: '🏃🏻‍♂️' if date in [datetime.strptime(d, '%Y-%m-%d').strftime('%d-%m-%y') for d in x] else ''
    )

# Verificar se o tempo_total de cada dia é maior que 30 minutos e adicionar 'tempo ok'
for date in date_range:
    grouped[date] = grouped.apply(
        lambda row: f"{row[date]} ⏱️" if row[date] == '🏃🏻‍♂️' and row['tempo_total'] > 30 else row[date], axis=1
    )

# Remover a coluna 'datas'
grouped = grouped.drop(columns=['datas'])
# Ordenar por total_quilometros em ordem decrescente
grouped = grouped.sort_values(by='total_quilometros', ascending=False)

# Redefinir o índice e adicionar uma coluna de numeração começando de 1
grouped.reset_index(drop=True, inplace=True)
grouped.index += 1
grouped.index.name = 'Classificação'


# Renomear a coluna 'total_quilometros' para 'Distância'
grouped.rename(columns={'total_quilometros': 'Distância'}, inplace=True)
grouped.rename(columns={'athlete.fullname': 'Atleta'}, inplace=True)
grouped.rename(columns={'tempo_total': 'Tempo Total'}, inplace=True)



pontuacao_participantes = grouped

pontuacao_participantes




,Atleta,Distância,Tempo Total,08-01-25,09-01-25,10-01-25,11-01-25,12-01-25,13-01-25,14-01-25,15-01-25,16-01-25,17-01-25,18-01-25
Classificação,,,,,,,,,,,,,,
1,RUY BARBOSA R.,44.04,231.53,🏃🏻‍♂️ ⏱️,,,,,,,,,,
2,Neto F.,31.10,201.55,🏃🏻‍♂️ ⏱️,,,,,,,,,,
3,Vanilda C.,25.37,175.41,🏃🏻‍♂️ ⏱️,,,,,,,,,,
4,Lara M.,23.02,153.90,🏃🏻‍♂️ ⏱️,,,,,,,,,,
5,Rogerio C.,21.58,119.27,🏃🏻‍♂️ ⏱️,,,,,,,,,,
6,Carlos C.,20.76,108.45,🏃🏻‍♂️ ⏱️,,,,,,,,,,
7,Tiago P.,19.70,102.03,🏃🏻‍♂️ ⏱️,,,,,,,,,,
8,Marcio C.,19.08,112.86,🏃🏻‍♂️ ⏱️,,,,,,,,,,
9,Jainne A.,18.21,112.32,🏃🏻‍♂️ ⏱️,,,,,,,,,,
